In [1]:
!pip install pyDOE
!pip install cma

  Created wheel for pyDOE: filename=pyDOE-0.3.8-cp36-none-any.whl size=18178 sha256=043a1aa3ba07883cdd51be51d3d40bdd10340628f9863a3c6015cc9860381237
  Stored in directory: /root/.cache/pip/wheels/7c/c8/58/a6493bd415e8ba5735082b5e0c096d7c1f2933077a8ce34544
Successfully built pyDOE
     |████████████████████████████████| 245kB 3.4MB/s 


In [0]:
import pyDOE
from scipy import stats
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds
from collections import namedtuple
import cma
from sklearn.preprocessing import MinMaxScaler
import scipy.stats.distributions as dist
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
import bbobbenchmarks as bn
import sklearn
from sklearn.preprocessing import PolynomialFeatures

## Surrogate Models & Helper Functions

In [0]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' F13 '''
def F13(X):
    f = bn.F13()
    X = np.array(X)
    return f(X)

''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs, dim):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=dim, samples=n_obs, criterion='m')
    X = [lhd[:,idx] for idx in range(dim)]
    return X

def create_basis_function(data):
    true = np.array(data['Y'])
    data = pd.DataFrame(np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(data.iloc[:,:-1])))
    data['Y'] = pd.Series(true)
    return data


''' Create Basis Functions '''
def create_function_basis(x):
    return np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(x.reshape(1,-1)))


''' Elastic Net Regression '''
def elastic_net(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    regr = ElasticNet(alpha= 0.04  ,random_state=0 , l1_ratio=1.0, fit_intercept =True, max_iter=3000,selection='random').fit(scaler.transform ( np.array(train_data.iloc[:,:-1])) ,  np.array(train_data.iloc[:,-1]))
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = create_function_basis(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)
    

''' Kriging'''
def kriging(train_data,test_data):
    kernel =  RBF()
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer= 15,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)


''' Support Vector Regression'''
def _SVR(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = sklearn.svm.SVR(kernel='rbf', gamma = 40.53 , C = 980.40 ,max_iter=1500).fit( scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

## Load Training and Test Data Set initially Generated

In [0]:
path = "train_13_4000Samples.csv"
train = pd.read_csv(path).iloc[:,1:]
test = pd.read_csv('test_13_800Samples.csv').iloc[:,1:]
true = np.array(test['Y'])

## Surrogate Models

In [5]:
model_kri , pred_kri , predict_kri = kriging(train,test.iloc[:,:-1])
model_svr , pred_svr , predict_svr = _SVR(train,test.iloc[:,:-1])
train = create_basis_function(train)
test = create_basis_function(test)
model_eln , pred_eln , predict_eln = elastic_net(train,test.iloc[:,:-1])

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


## CMA-ES

In [6]:
Columns = ['Kri' , 'SVR' , 'ELN' ]
Cols = []
for j in range(len(Columns)):
    for i in range(1,201):
        Cols.append(Columns[j]+'_X'+str(i))
const = [ [-5] * 200, [5] * 200 ]

opt = cma.CMAOptions()
opt.set("bounds", const)
opt.set ("seed" , 0)
opt.set ("maxfevals" , 200000)

n_obs , dim =  30, 200
G = DOE(n_obs, dim)
G  = [ linearscaletransform(G[idx] , range_out=(-5,5)) for idx in range(dim) ]
G = [ G[idx].reshape(n_obs,1) for idx in range(len(G)) ]
X_Values = np.zeros([10,600])
for i in range(10, 20):
  print ('Run : '+ str(i))
  min_kri = cma.fmin(predict_kri , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  min_svr = cma.fmin(predict_svr , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  min_eln = cma.fmin(predict_eln , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  X_Values [i-10,:] = list(min_kri)+list(min_svr)+list(min_eln)

Run : 10
(9_w,19)-aCMA-ES (mu_w=5.6,w_1=29%) in dimension 200 (seed=285919, Sat Apr 18 14:17:59 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     19 1.050368564394253e+04 1.0e+00 2.44e+00  2e+00  2e+00 0:00.1
    2     38 1.028503644192891e+04 1.0e+00 2.38e+00  2e+00  2e+00 0:00.3


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [10971.24557137] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [10999.56167802] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


    3     57 1.026417623265991e+04 1.0e+00 2.34e+00  2e+00  2e+00 0:00.4
   27    513 7.517419051536082e+03 1.1e+00 2.16e+00  2e+00  2e+00 0:03.4
   59   1121 7.343760914831369e+03 1.1e+00 2.06e+00  2e+00  2e+00 0:07.5
   99   1881 6.530646087511524e+03 1.2e+00 1.71e+00  2e+00  2e+00 0:12.5
  100   1900 6.186565344277007e+03 1.2e+00 1.69e+00  2e+00  2e+00 0:12.6
  154   2926 5.103408927095694e+03 1.2e+00 1.07e+00  1e+00  1e+00 0:19.6
  200   3800 3.993624919263691e+03 1.2e+00 6.80e-01  7e-01  7e-01 0:25.7
  270   5130 3.087086522377695e+03 1.2e+00 3.08e-01  3e-01  3e-01 0:34.7
  300   5700 2.976515281495475e+03 1.2e+00 2.24e-01  2e-01  2e-01 0:38.6
  387   7353 2.831629025881488e+03 1.2e+00 8.50e-02  8e-02  9e-02 0:49.7
  400   7600 2.824592075789949e+03 1.2e+00 7.33e-02  7e-02  7e-02 0:51.4
  500   9500 2.808840650935006e+03 1.2e+00 2.48e-02  2e-02  2e-02 1:03.9
  600  11400 2.807204368445794e+03 1.3e+00 8.12e-03  8e-03  8e-03 1:16.7
  700  13300 2.807033076748668e+03 1.3e+00 2.65e-03

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [10379.69225751] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [10728.5403757] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


    2     38 1.042242441427101e+04 1.0e+00 2.38e+00  2e+00  2e+00 0:00.2
    3     57 9.764093216344518e+03 1.0e+00 2.34e+00  2e+00  2e+00 0:00.3


...

   25    475 7.550209423820570e+03 1.1e+00 2.12e+00  2e+00  2e+00 0:03.5
   54   1026 6.988768159748704e+03 1.1e+00 1.99e+00  2e+00  2e+00 0:07.6
   89   1691 6.534719748643269e+03 1.2e+00 1.63e+00  2e+00  2e+00 0:12.7
  100   1900 6.487659313305806e+03 1.2e+00 1.49e+00  1e+00  1e+00 0:14.3
  149   2831 4.989494451124303e+03 1.2e+00 9.62e-01  1e+00  1e+00 0:21.3
  200   3800 3.958715574664049e+03 1.2e+00 6.00e-01  6e-01  6e-01 0:28.6
  264   5016 3.082999237119760e+03 1.2e+00 2.89e-01  3e-01  3e-01 0:37.7
  300   5700 2.913635076044141e+03 1.2e+00 1.92e-01  2e-01  2e-01 0:42.7
  379   7201 2.825965464193367e+03 1.2e+00 8.00e-02  8e-02  8e-02 0:53.9
  400   7600 2.820660996887595e+03 1.2e+00 6.29e-02  6e-02  6e-02 0:56.8
  492   9348 2.808912762814228e+03 1.2e+00 2.35e-02  2e-02  2e-02 1:09.8
  500   9500 2.808715089756795e+03 1.2e+00 2.16e-02  2e-02  2e-02 1:11.0
  600  11400 2.807207252123680e+03 1.2e+00 7.95e-03  8e-03  8e-03 1:25.4
  700  13300 2.807034687376745e+03 1.2e+00 2.67e-03

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [10379.69225751] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [10022.28683868] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


    2     38 9.433850783787244e+03 1.0e+00 2.39e+00  2e+00  2e+00 0:00.2
    3     57 9.234773114764983e+03 1.0e+00 2.35e+00  2e+00  2e+00 0:00.3


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [10773.77866832] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


   34    646 6.361010564913635e+03 1.1e+00 2.01e+00  2e+00  2e+00 0:03.4
   75   1425 5.765279128565802e+03 1.2e+00 1.69e+00  2e+00  2e+00 0:07.5
  100   1900 5.236565101719738e+03 1.2e+00 1.42e+00  1e+00  1e+00 0:09.9
  162   3078 4.395895696270994e+03 1.2e+00 8.60e-01  9e-01  9e-01 0:15.9
  200   3800 4.081131203234954e+03 1.2e+00 6.13e-01  6e-01  6e-01 0:19.6
  284   5396 3.784932386321177e+03 1.2e+00 3.14e-01  3e-01  3e-01 0:27.7
  300   5700 3.728638680454780e+03 1.2e+00 2.83e-01  3e-01  3e-01 0:29.2
  400   7600 3.619601381595498e+03 1.2e+00 1.30e-01  1e-01  1e-01 0:38.8
  500   9500 3.590305220077849e+03 1.2e+00 7.16e-02  7e-02  7e-02 0:48.4
  600  11400 3.580952024804746e+03 1.3e+00 4.30e-02  4e-02  4e-02 0:58.0
  700  13300 3.577727942742908e+03 1.4e+00 2.50e-02  2e-02  3e-02 1:07.6
  800  15200 3.575753155075645e+03 1.6e+00 1.88e-02  2e-02  2e-02 1:17.3
  900  17100 3.574096797451552e+03 2.0e+00 2.19e-02  2e-02  3e-02 1:26.9
 1000  19000 3.572175588366545e+03 2.5e+00 2.34e-02

In [7]:
X_Values = pd.DataFrame(X_Values)
X_Values.columns = Cols
X_Values.to_csv('sample_data\\F13_X_Values_first.csv')
Krig_Fun = np.zeros(10)
SVM_Fun = np.zeros(10)
ELN_Fun = np.zeros(10)
for i in range(X_Values.shape[0]):
    Krig_Fun [i] = F13(X_Values.iloc[i,:200])
    SVM_Fun [i] = F13(X_Values.iloc[i,200:400])
    ELN_Fun [i] = F13(X_Values.iloc[i,400:600])
print ('Kriging')
print (stats.mode(Krig_Fun) , np.std(Krig_Fun))
print ('SVM')
print (stats.mode(SVM_Fun) , np.std(SVM_Fun))
print ('ELN')
print (stats.mode(ELN_Fun) , np.std(ELN_Fun))

Kriging
ModeResult(mode=array([2858.18444392]), count=array([1])) 2.0995800643003117e-05
SVM
ModeResult(mode=array([9462.46927809]), count=array([1])) 714.5847347910725
ELN
ModeResult(mode=array([7428.35337331]), count=array([1])) 23.53474444039935
